In [1]:
## import packages 

from bs4 import BeautifulSoup as Soup
import requests
import pandas as pd
import numpy as np
import pulp
import re

In [2]:
## set constraints 

## Update these 
auction_dollars = 194.00
pos_drafted = {
    "QB": 0,
    "RB": 0,
    "WR": 0,
    "TE": 0
}

## leave these alone
pos_max = {
    "QB": 1,
    "RB": 3,
    "WR": 3,
    "TE": 2

}

pos_min = {
    "QB": 1,
    "RB": 2,
    "WR": 2,
    "TE": 1
}

starter_max = 7.00 - sum(pos_drafted.values())


In [3]:
## import data frame from excel
excel_path = '/Users/cameronfryzel/Documents/FantasyFootball/FF_AuctionDraft_2023.xlsx'

dict_df = pd.read_excel(excel_path, sheet_name = ['QB', 'RB', 'WR', 'TE'])

In [4]:
## Convert dicts to df's and union 

df_qb = dict_df.get('QB')
df_rb = dict_df.get('RB')
df_wr = dict_df.get('WR')
df_te = dict_df.get('TE')

df_final = pd.concat([df_wr, df_rb, df_qb, df_te])
df_final.reset_index(inplace=True)


In [5]:
## Aggregate objective function set up

##create empty dictionaries
values = {}
points = {}
##iterate through unique positions
for pos in df_final.position.unique():
    ## create temp df of each position
    available_pos = df_final[df_final.position == pos]
    ## create k/v pairs for player and auction value
    value = list(available_pos[["name","auction_value_ds"]].set_index("name").to_dict().values())[0]
    point = list(available_pos[["name","ds_points"]].set_index("name").to_dict().values())[0]
    ## insert into dicts
    values[pos] = value
    points[pos] = point

In [6]:
## idk what this is doing (I think creating variables for each k/v pair? so WR_Justin_Jefferson becomes a variable to
## get a 1 or 0 binary?)
_vars = {k: pulp.LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

In [7]:
##Surplus objective function set up
##create empty dictionaries
values_s = {}
points_s = {}
##iterate through unique positions
for pos in df_final.position.unique():
    ## create temp df of each position
    available_pos = df_final[df_final.position == pos]
    ## create k/v pairs for player and auction value
    value_s = list(available_pos[["name","surplus_auction_value"]].set_index("name").to_dict().values())[0]
    point_s = list(available_pos[["name","ds_points"]].set_index("name").to_dict().values())[0]
    ## insert into dicts
    values_s[pos] = value_s
    points_s[pos] = point_s
    
_vars_s = {k: pulp.LpVariable.dict(k, v, cat="Binary") for k, v in points_s.items()}

In [8]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        for constraint_2 in constraints:
            c = re.findall("[0-9\.]*\*1.0", constraint_2)
        if v.varValue != 0:
            print(v.name, "=", c[-1])
            ##print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [9]:
## DataSharks Auction Value 

## name our problem MaxFF and state we want to maximize something (points)
prob_ds = pulp.LpProblem("Fantasy_DS", pulp.LpMaximize)

#set the empty lists
max_points_ds = []
costs_ds = []
position_constraints_ds = []


for k, v in _vars.items():
    ## sum of player costs
    costs_ds += pulp.lpSum([values[k][i] * _vars[k][i] for i in v])
    ## sum of points
    max_points_ds += pulp.lpSum([points[k][i] * _vars[k][i] for i in v])
    ## constraints on players
    prob_ds += pulp.lpSum([_vars[k][i] for i in v]) <= (pos_max[k] - pos_drafted[k])
    prob_ds += pulp.lpSum([_vars[k][i] for i in v]) >= (pos_min[k] - pos_drafted[k])
    ##for i in v:
    ##    prob_all += pulp.lpSum(_vars[k][i]) <= starter_max
    position_constraints_ds += pulp.lpSum([v[i] for i in v]) 

## maximize points and keep costs below auction dollars
prob_ds += pulp.lpSum(max_points_ds)
prob_ds += pulp.lpSum(position_constraints_ds) <= starter_max
prob_ds += pulp.lpSum(costs_ds) <= auction_dollars


In [10]:
## Surplus Auction Value 

## name our problem MaxFF and state we want to maximize something (points)
prob = pulp.LpProblem("Fantasy_Surplus", pulp.LpMaximize)

#set the empty lists
max_points = []
costs = []
position_constraints = []


for k, v in _vars_s.items():
    ## sum of player costs
    costs += pulp.lpSum([values_s[k][i] * _vars_s[k][i] for i in v])
    ## sum of points
    max_points += pulp.lpSum([points_s[k][i] * _vars_s[k][i] for i in v])
    ## constraints on players
    prob += pulp.lpSum([_vars_s[k][i] for i in v]) <= (pos_max[k] - pos_drafted[k])
    prob += pulp.lpSum([_vars_s[k][i] for i in v]) >= (pos_min[k] - pos_drafted[k])
    ##for i in v:
    ##    prob_all += pulp.lpSum(_vars[k][i]) <= starter_max
    position_constraints += pulp.lpSum([v[i] for i in v]) 

## maximize points and keep costs below auction dollars
prob += pulp.lpSum(max_points)
prob += pulp.lpSum(position_constraints) <= starter_max
prob += pulp.lpSum(costs) <= auction_dollars

In [11]:
## Solve based on Draft Sharks Value

prob_ds.solve()
summary(prob_ds)

Variables:

QB__Josh_Allen = 41.56480938416423*1.0
RB__D'Andre_Swift = 1.586802946039391*1.0
RB__Khalil_Herbert = 4.283330828197819*1.0
TE__Travis_Kelce = 36.25361512791991*1.0
WR__Garrett_Wilson = 29.28697835704891*1.0
WR__Jaylen_Waddle = 24.11658495755112*1.0
WR__Tyreek_Hill = 56.87157718522061*1.0
---------------------------------------

Constraints:
41.56480938416423*1.0 + 1.586802946039391*1.0 + 4.283330828197819*1.0 + 36.25361512791991*1.0 + 29.28697835704891*1.0 + 24.11658495755112*1.0 + 56.87157718522061*1.0 = 193.963698786142
---------------------------------------

Score:
415.7*1.0 + 147.25*1.0 + 151.15*1.0 + 226.7*1.0 + 212.0*1.0 + 202.6*1.0 + 262.15*1.0 = 1617.5499999999997


In [12]:
## Solve based on Surplus Auction Value

prob.solve()
summary(prob)

Variables:

QB__Justin_Herbert = 30.71636263688574*1.0
RB__A.J._Dillon = 4.574348662606519*1.0
RB__Austin_Ekeler = 57.45715856322421*1.0
TE__Pat_Freiermuth = 12.63775369971212*1.0
WR__Chris_Godwin = 7.050081733620729*1.0
WR__Drake_London = 9.324301647691925*1.0
WR__Justin_Jefferson = 72.22519420776251*1.0
---------------------------------------

Constraints:
30.71636263688574*1.0 + 4.574348662606519*1.0 + 57.45715856322421*1.0 + 12.63775369971212*1.0 + 7.050081733620729*1.0 + 9.324301647691925*1.0 + 72.22519420776251*1.0 = 193.98520115150376
---------------------------------------

Score:
342.5*1.0 + 152.9*1.0 + 244.75*1.0 + 132.25*1.0 + 171.0*1.0 + 174.95*1.0 + 284.2*1.0 = 1502.5500000000002
